In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
from scipy.stats import iqr # For Finding Volume Outliers

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from nsepython import *

In [60]:
## Stock Symbols contains list of 1250 NSE stock symbols (Need to update it every quarter / year)

stock_data = pd.read_csv('stock_data.csv')
stock_data.set_index('SYMBOL', inplace=True)
len(stock_data)

1435

In [70]:
# Fetch Previous Bulk Deals from the Bulk Deals file
bulk_deals = pd.read_csv('Bulk_Deals.csv')
bulk_deals.set_index('SYMBOL', inplace=True)

In [128]:
# Fetch New Bulk Deals from the NSE
date = pd.to_datetime('today')
for i in range(1, 2):
    from_date = date + pd.DateOffset(-(4-i))
    date_str = from_date.strftime("%d-%m-%Y")
    print(date_str)
    new_bulk_deals = nse_largedeals_historical(date_str, date_str)
    if new_bulk_deals.shape[0] > 0:
        new_bulk_deals = new_bulk_deals[new_bulk_deals['BD_CLIENT_NAME'].isin(['GRAVITON RESEARCH CAPITAL LLP','HRTI PRIVATE LIMITED'])]
        new_bulk_deals = new_bulk_deals[['BD_SYMBOL']]
        new_bulk_deals.drop_duplicates(inplace=True)

        for symbol in new_bulk_deals['BD_SYMBOL'].values:
            if symbol in bulk_deals.index:
                bulk_deals.loc[symbol]['COUNT'] += 1
                bulk_deals.loc[symbol]['DATE'] = date_str
            else:
                new_deal = {'COUNT': 1, 'DATE': date_str}
                bulk_deals.loc[symbol] = new_deal
bulk_deals = bulk_deals.sort_values('DATE', ascending=False)

24-01-2024
{'data': [{'_id': '65b108a716e02b4d67f9df85', 'BD_DT_DATE': '24-JAN-2024', 'BD_SYMBOL': 'AGNI', 'BD_SCRIP_NAME': 'Agni Green Power Ltd', 'BD_CLIENT_NAME': 'YUGA STOCKS AND COMMODITIES PRIVATE LIMITED  .', 'BD_BUY_SELL': 'BUY', 'BD_QTY_TRD': 100000, 'BD_TP_WATP': 52.2, 'BD_REMARKS': '-', 'TIMESTAMP': '2024-01-23T18:30:00.000Z', 'createdAt': '2024-01-24T12:55:03.477Z', 'updatedAt': '2024-01-24T12:55:03.477Z', '__v': 0, 'mTIMESTAMP': '24-Jan-2024'}, {'_id': '65b108a716e02b4d67f9df86', 'BD_DT_DATE': '24-JAN-2024', 'BD_SYMBOL': 'AKANKSHA', 'BD_SCRIP_NAME': 'Akanksha Power N Infra L', 'BD_CLIENT_NAME': 'MULTIPLIER SHARE & STOCK ADVISORS PRIVATE LIMITED', 'BD_BUY_SELL': 'BUY', 'BD_QTY_TRD': 124000, 'BD_TP_WATP': 135.85, 'BD_REMARKS': '-', 'TIMESTAMP': '2024-01-23T18:30:00.000Z', 'createdAt': '2024-01-24T12:55:03.477Z', 'updatedAt': '2024-01-24T12:55:03.477Z', '__v': 0, 'mTIMESTAMP': '24-Jan-2024'}, {'_id': '65b108a716e02b4d67f9dfc8', 'BD_DT_DATE': '24-JAN-2024', 'BD_SYMBOL': 'AKANK

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_6693/3648612613.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_6693/3648612613.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Save The File Now

In [106]:
bulk_deals.to_csv('Bulk_Deals.csv')

In [109]:
figure_html = open('bulk_deals_figure.html', 'w')

plot_symbols = [symbol for symbol in bulk_deals.index.values if symbol in stock_data.index.values]

for symbol in plot_symbols:
    data = yf.download(symbol + ".NS", start='2023-01-01', end=pd.to_datetime('today')+pd.DateOffset(1), progress=False)
    
    data['20DMA'] = data['Close'].rolling(window=20).mean()
    data['40DMA']= data['Close'].rolling(window=40).mean()
    data['100DMA']= data['Close'].rolling(window=100).mean()
    
    data['diff'] = data['Close'] - data['Open']
    data.loc[data['diff'] >= 0, 'color'] = 'green'
    data.loc[data['diff'] < 0, 'color'] = 'red'
    
    plot_data = data[data.index > (pd.to_datetime('today') - pd.DateOffset(days=240))]

    figure = make_subplots(specs=[[{"secondary_y": True}]])
    figure.add_trace(go.Candlestick(x = plot_data.index,
                              open = plot_data['Open'],
                              high = plot_data['High'],
                              low = plot_data['Low'],
                              close = plot_data['Close'],
                              name='Price'))
    # Adding Text Below
    stock_row = stock_data.loc[symbol]
    figure.add_annotation(dict(font=dict(color='black', size=16.5),
        x=0.5,  # Center aligned horizontally
        y=-0.12, showarrow=False,
        text="SECTOR - " + str(stock_row['SECTOR']) + "  FNO - " + str(bool(stock_row['FNO'])),
        textangle=0,
        xanchor='center',  # Center aligned horizontally
        yanchor='bottom',  # Aligned to the bottom
        xref="paper", yref="paper"))
    
    figure.add_trace(go.Scatter(x=plot_data.index, y=plot_data['20DMA'], marker_color='blue',name='20 Day MA'))
    figure.add_trace(go.Scatter(x=plot_data.index, y=plot_data['40DMA'], marker_color='orange',name='40 Day MA'))
    figure.add_trace(go.Scatter(x=plot_data.index, y=plot_data['100DMA'], marker_color='green',name='100 Day MA'))
    figure.add_trace(go.Bar(x=plot_data.index, y=plot_data['Volume'], name='Volume', marker={'color':plot_data['color']}),secondary_y=True)
    
    figure.update_yaxes(range=[0, plot_data['Volume'].max()*5], secondary_y=True)
    figure.update_yaxes(visible=False, secondary_y=True)
    figure.update_layout(title={'text':symbol, 'x':0.5})
    
    figure.update_layout(xaxis_rangeslider_visible=False)  #hide range slider
    figure.update_xaxes(rangebreaks = [ dict(bounds=['sat','mon']) ] ) # hide weekends 
    
    # figure.show()
    figure_html.write(figure.to_html(full_html=False))
print('Execution Completed!!!!')

Execution Completed!!!!


In [105]:
bulk_deals

,COUNT,DATE
SYMBOL,,
INFIBEAM,1,25-01-2024
GMBREW,1,04-01-2024
HFCL,1,04-01-2024
COASTCORP,1,04-01-2024
BRNL,1,04-01-2024
...,...,...
MKPL,1,15-01-2024
INTENTECH,1,16-01-2024
EKC,1,16-01-2024


In [108]:
bulk_deals = bulk_deals.sort_values('DATE', ascending=False)

In [127]:
new_bulk_deals = nse_largedeals_historical('19-01-2024', '19-01-2024')

{'data': [{'_id': '65aa6c76f173ca68878c32ce', 'BD_DT_DATE': '19-JAN-2024', 'BD_SYMBOL': 'AARTISURF', 'BD_SCRIP_NAME': 'Aarti Surfactants Limited', 'BD_CLIENT_NAME': 'GRAVITON RESEARCH CAPITAL LLP', 'BD_BUY_SELL': 'BUY', 'BD_QTY_TRD': 50311, 'BD_TP_WATP': 750.09, 'BD_REMARKS': '-', 'TIMESTAMP': '2024-01-18T18:30:00.000Z', 'createdAt': '2024-01-19T12:35:02.906Z', 'updatedAt': '2024-01-19T12:35:02.906Z', '__v': 0, 'mTIMESTAMP': '19-Jan-2024'}, {'_id': '65aa6c76f173ca68878c3313', 'BD_DT_DATE': '19-JAN-2024', 'BD_SYMBOL': 'AARTISURF', 'BD_SCRIP_NAME': 'Aarti Surfactants Limited', 'BD_CLIENT_NAME': 'GRAVITON RESEARCH CAPITAL LLP', 'BD_BUY_SELL': 'SELL', 'BD_QTY_TRD': 50311, 'BD_TP_WATP': 750.35, 'BD_REMARKS': '-', 'TIMESTAMP': '2024-01-18T18:30:00.000Z', 'createdAt': '2024-01-19T12:35:02.908Z', 'updatedAt': '2024-01-19T12:35:02.908Z', '__v': 0, 'mTIMESTAMP': '19-Jan-2024'}, {'_id': '65aa6c76f173ca68878c32cf', 'BD_DT_DATE': '19-JAN-2024', 'BD_SYMBOL': 'AKANKSHA', 'BD_SCRIP_NAME': 'Akanksha Po

In [126]:
new_bulk_deals.iloc[50:150]

,_id,BD_DT_DATE,BD_SYMBOL,BD_SCRIP_NAME,BD_CLIENT_NAME,BD_BUY_SELL,BD_QTY_TRD,BD_TP_WATP,BD_REMARKS,TIMESTAMP,createdAt,updatedAt,__v,mTIMESTAMP
50,65aa6c7960f82ae6d1c6693a,19-JAN-2024,LIBERTSHOE,Liberty Shoes Ltd,HRTI PRIVATE LIMITED,BUY,199660,370.97,-,2024-01-18T18:30:00.000Z,2024-01-19T12:35:05.351Z,2024-01-19T12:35:05.351Z,0,19-Jan-2024
51,65aa6c7960f82ae6d1c66982,19-JAN-2024,LIBERTSHOE,Liberty Shoes Ltd,HRTI PRIVATE LIMITED,SELL,214389,370.38,-,2024-01-18T18:30:00.000Z,2024-01-19T12:35:05.352Z,2024-01-19T12:35:05.352Z,0,19-Jan-2024
52,65aa6c7960f82ae6d1c66939,19-JAN-2024,LIBERTSHOE,Liberty Shoes Ltd,SOCIETE GENERALE,BUY,105711,370.78,-,2024-01-18T18:30:00.000Z,2024-01-19T12:35:05.351Z,2024-01-19T12:35:05.351Z,0,19-Jan-2024
53,65aa6c7960f82ae6d1c6693c,19-JAN-2024,MIRZAINT,Mirza International Ltd.,GRAVITON RESEARCH CAPITAL LLP,BUY,1885345,61.72,-,2024-01-18T18:30:00.000Z,2024-01-19T12:35:05.351Z,2024-01-19T12:35:05.351Z,0,19-Jan-2024
54,65aa6c7960f82ae6d1c66985,19-JAN-2024,MIRZAINT,Mirza International Ltd.,GRAVITON RESEARCH CAPITAL LLP,SELL,1885345,61.68,-,2024-01-18T18:30:00.000Z,2024-01-19T12:35:05.352Z,2024-01-19T12:35:05.352Z,0,19-Jan-2024
55,65aa6c7960f82ae6d1c6693e,19-JAN-2024,MIRZAINT,Mirza International Ltd.,HRTI PRIVATE LIMITED,BUY,1026323,61.65,-,2024-01-18T18:30:00.000Z,2024-01-19T12:35:05.351Z,2024-01-19T12:35:05.351Z,0,19-Jan-2024
56,65aa6c7960f82ae6d1c66984,19-JAN-2024,MIRZAINT,Mirza International Ltd.,HRTI PRIVATE LIMITED,SELL,977050,61.52,-,2024-01-18T18:30:00.000Z,2024-01-19T12:35:05.352Z,2024-01-19T12:35:05.352Z,0,19-Jan-2024
57,65aa6c7960f82ae6d1c6693d,19-JAN-2024,MIRZAINT,Mirza International Ltd.,SKI CAPITAL SERVICES LTD,BUY,1028023,61.34,-,2024-01-18T18:30:00.000Z,2024-01-19T12:35:05.351Z,2024-01-19T12:35:05.351Z,0,19-Jan-2024
58,65aa6c7960f82ae6d1c66986,19-JAN-2024,MIRZAINT,Mirza International Ltd.,SKI CAPITAL SERVICES LTD,SELL,1124023,61.43,-,2024-01-18T18:30:00.000Z,2024-01-19T12:35:05.352Z,2024-01-19T12:35:05.352Z,0,19-Jan-2024
59,65aa6c7960f82ae6d1c66988,19-JAN-2024,MITTAL,Mittal Life Style Limited,SHOBA DEVI YADAV,SELL,1950000,2.25,-,2024-01-18T18:30:00.000Z,2024-01-19T12:35:05.352Z,2024-01-19T12:35:05.352Z,0,19-Jan-2024
